In [50]:
import sys
#sys.path.insert(0, "../..")

from typing import Callable, Protocol, Dict, Optional, Iterator, List, Tuple

import gin
from pathlib import Path
import numpy as np
import math
from datetime import datetime

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Optimizer
from torchinfo import summary
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#from src.data import make_dataset
#from src.models import metrics
#from src.models import train_model # om de functie train_loop binnen te halen

# ------------------------------------------------------------------------------- #
## Alle imports benodigd voor de functie train_loop uit train_model.py 

import tensorflow as tf  # noqa: F401

# needed to make summarywriter load without error
from loguru import logger
from numpy import Inf
from ray import tune


from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

#from src.models.metrics import Metric
#from src.typehinting import GenericModel
#from src.data import data_tools

In [51]:
## benodigde functie uitmake_dataset.py

#@gin.configurable
def get_MNIST(  # noqa: N802
    data_dir: Path, batch_size: int
) -> Tuple[DataLoader, DataLoader]:

    training_data = datasets.FashionMNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

In [52]:
## Benodigde funties uit metrics.py

Tensor = torch.Tensor

class Metric:
    def __repr__(self) -> str:
        raise NotImplementedError

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        raise NotImplementedError

class Accuracy(Metric):
    def __repr__(self) -> str:
        return "Accuracy"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        return (yhat.argmax(dim=1) == y).sum() / len(yhat)


In [53]:
class GenericModel(Protocol):
    train: Callable
    eval: Callable
    parameters: Callable

    def __call__(self, *args, **kwargs) -> torch.Tensor:
        pass


In [54]:
def count_parameters(model: GenericModel) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [55]:
## uit data_tools.py

def dir_add_timestamp(log_dir: Optional[Path] = None) -> Path:
    if log_dir is None:
        log_dir = Path(".")
    log_dir = Path(log_dir)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    log_dir = log_dir / timestamp
    logger.info(f"Logging to {log_dir}")
    if not log_dir.exists():
        log_dir.mkdir(parents=True)
    return log_dir

In [56]:
## Definieren functie train_loop

#@gin.configurable
def trainloop(
    epochs: int,
    model: GenericModel,
    optimizer: torch.optim.Optimizer,
    learning_rate: float,
    loss_fn: Callable,
    metrics: List[Metric],
    train_dataloader: Iterator,
    test_dataloader: Iterator,
    log_dir: Path,
    train_steps: int,
    eval_steps: int,
    patience: int = 10,
    factor: float = 0.9,
    tunewriter: bool = False,
) -> GenericModel:
    
    optimizer_: torch.optim.Optimizer = optimizer(
        model.parameters(), lr=learning_rate
    )  # type: ignore

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_,
        factor=factor,
        patience=patience,
    )

    if not tunewriter:
        log_dir = dir_add_timestamp(log_dir)
        writer = SummaryWriter(log_dir=log_dir)
        #write_gin(log_dir, gin.config_str())

        images, _ = next(iter(train_dataloader))
        if len(images.shape) == 4:
            grid = make_grid(images)
            writer.add_image("images", grid)
        writer.add_graph(model, images)

    for epoch in tqdm(range(epochs)):
        train_loss = trainbatches(
            model, train_dataloader, loss_fn, optimizer_, train_steps
        )

        metric_dict, test_loss = evalbatches(
            model, test_dataloader, loss_fn, metrics, eval_steps
        )

        scheduler.step(test_loss)

        if tunewriter:
            tune.report(
                iterations=epoch,
                train_loss=train_loss,
                test_loss=test_loss,
                **metric_dict,
            )
        else:
            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            for m in metric_dict:
                writer.add_scalar(f"metric/{m}", metric_dict[m], epoch)
            lr = [group["lr"] for group in optimizer_.param_groups][0]
            writer.add_scalar("learning_rate", lr, epoch)
            metric_scores = [f"{v:.4f}" for v in metric_dict.values()]
            logger.info(
                f"Epoch {epoch} train {train_loss:.4f} test {test_loss:.4f} metric {metric_scores}"  # noqa E501
            )

    return model

In [57]:
## uit train_model.py

def trainbatches(
    model: GenericModel,
    traindatastreamer: Iterator,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    train_steps: int,
) -> float:
    model.train()
    train_loss: float = 0.0
    for _ in tqdm(range(train_steps)):
        x, y = next(iter(traindatastreamer))
        optimizer.zero_grad()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.detach().numpy()
    train_loss /= train_steps
    return train_loss

In [58]:
## uit train_model.py

def evalbatches(
    model: GenericModel,
    testdatastreamer: Iterator,
    loss_fn: Callable,
    metrics: List[Metric],
    eval_steps: int,
) -> Tuple[Dict[str, float], float]:
    model.eval()
    test_loss: float = 0.0
    metric_dict: Dict[str, float] = {}
    for _ in range(eval_steps):
        x, y = next(iter(testdatastreamer))
        yhat = model(x)
        test_loss += loss_fn(yhat, y).detach().numpy()
        for m in metrics:
            metric_dict[str(m)] = (
                metric_dict.get(str(m), 0.0) + m(y, yhat).detach().numpy()
            )

    test_loss /= eval_steps
    for key in metric_dict:
        metric_dict[str(key)] = metric_dict[str(key)] / eval_steps
    return metric_dict, test_loss


In [59]:
#gin.parse_config_file("model.gin")

In [60]:
#sys.path

In [61]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [62]:
## Model 1: CNN_een met een convolutional layer & drie linear layers

#@gin.configurable
class CNN_een_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6272, 3136),
            nn.ReLU(),
            nn.Linear(3136, 1568),
            nn.ReLU(),
            nn.Linear(1568, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_een_drie = CNN_een_drie(10, 3, 32, 32).to(device)

In [63]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers

#@gin.configurable
class CNN_twee_twee(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_twee = CNN_twee_twee(10, 3, 32, 32).to(device)

In [78]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers & 16-32 filters

#@gin.configurable
class CNN_twee_twee_channels_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_twee_channels_16_32 = CNN_twee_twee_channels_16_32(10, 3, 16, 32).to(device)

print(summary(model_twee_twee_channels_16_32, input_size=(64, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_twee_twee_channels_16_32             [64, 10]                  --
├─Sequential: 1-1                        [64, 32, 6, 6]            --
│    └─Conv2d: 2-1                       [64, 16, 28, 28]          160
│    └─ReLU: 2-2                         [64, 16, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 16, 14, 14]          --
│    └─Conv2d: 2-4                       [64, 32, 12, 12]          4,640
│    └─ReLU: 2-5                         [64, 32, 12, 12]          --
│    └─MaxPool2d: 2-6                    [64, 32, 6, 6]            --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-7                      [64, 1152]                --
│    └─Linear: 2-8                       [64, 576]                 664,128
│    └─ReLU: 2-9                         [64, 576]                 --
│    └─Linear: 2-10                      [64, 10]                  5,770
Tot

In [77]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers & 32-64 filters

#@gin.configurable
class CNN_twee_twee_channels_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_twee_channels_32_64 = CNN_twee_twee_channels_32_64(10, 3, 32, 64).to(device)

print(summary(model_twee_twee_channels_32_64, input_size=(64, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_twee_twee_channels_32_64             [64, 10]                  --
├─Sequential: 1-1                        [64, 64, 6, 6]            --
│    └─Conv2d: 2-1                       [64, 32, 28, 28]          320
│    └─ReLU: 2-2                         [64, 32, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 32, 14, 14]          --
│    └─Conv2d: 2-4                       [64, 64, 12, 12]          18,496
│    └─ReLU: 2-5                         [64, 64, 12, 12]          --
│    └─MaxPool2d: 2-6                    [64, 64, 6, 6]            --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-7                      [64, 2304]                --
│    └─Linear: 2-8                       [64, 1152]                2,655,360
│    └─ReLU: 2-9                         [64, 1152]                --
│    └─Linear: 2-10                      [64, 10]                  11,530

In [66]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

#@gin.configurable
class CNN_twee_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_drie = CNN_twee_drie(10, 3, 32, 32).to(device)

In [76]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

#@gin.configurable
class CNN_twee_drie_channels_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_drie_channels_16_32 = CNN_twee_drie_channels_16_32(10, 3, 16, 32).to(device)

print(summary(model_twee_drie_channels_16_32, input_size=(64, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_twee_drie_channels_16_32             [64, 10]                  --
├─Sequential: 1-1                        [64, 32, 6, 6]            --
│    └─Conv2d: 2-1                       [64, 16, 28, 28]          160
│    └─ReLU: 2-2                         [64, 16, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 16, 14, 14]          --
│    └─Conv2d: 2-4                       [64, 32, 12, 12]          4,640
│    └─ReLU: 2-5                         [64, 32, 12, 12]          --
│    └─MaxPool2d: 2-6                    [64, 32, 6, 6]            --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-7                      [64, 1152]                --
│    └─Linear: 2-8                       [64, 576]                 664,128
│    └─ReLU: 2-9                         [64, 576]                 --
│    └─Linear: 2-10                      [64, 288]                 166,176
│

In [75]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

#@gin.configurable
class CNN_twee_drie_channels_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_drie_channels_32_64 = CNN_twee_drie_channels_32_64(10, 3, 32, 64).to(device)

print(summary(model_twee_drie_channels_32_64, input_size=(64, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_twee_drie_channels_32_64             [64, 10]                  --
├─Sequential: 1-1                        [64, 64, 6, 6]            --
│    └─Conv2d: 2-1                       [64, 32, 28, 28]          320
│    └─ReLU: 2-2                         [64, 32, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 32, 14, 14]          --
│    └─Conv2d: 2-4                       [64, 64, 12, 12]          18,496
│    └─ReLU: 2-5                         [64, 64, 12, 12]          --
│    └─MaxPool2d: 2-6                    [64, 64, 6, 6]            --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-7                      [64, 2304]                --
│    └─Linear: 2-8                       [64, 1152]                2,655,360
│    └─ReLU: 2-9                         [64, 1152]                --
│    └─Linear: 2-10                      [64, 576]                 664,12

In [69]:
## Model 2.4: CNN_twee met twee convolutional layers & vier linear layers

#@gin.configurable
class CNN_twee_vier(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 144),
            nn.ReLU(),
            nn.Linear(144, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_vier = CNN_twee_vier(10, 3, 32, 32).to(device)

In [70]:
## Model 3.2: CNN_drie met twee convolutional layers & twee linear layers

#@gin.configurable
class CNN_drie_twee(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_twee = CNN_drie_twee(10, 3, 32, 32).to(device)

In [71]:
## Model 3.3: CNN_drie met drie convolutional layers

#@gin.configurable
class CNN_drie_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_drie = CNN_drie_drie(10, 3, 32, 32).to(device)

In [72]:
## Model 3.4: CNN_drie met drie convolutional layers & vier linear layers

#@gin.configurable
class CNN_drie_vier(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_vier = CNN_drie_vier(10, 3, 32, 32).to(device)

In [73]:
modellen = [
            model_een_drie,
            model_twee_twee,
            model_twee_drie,
            model_twee_vier,
            model_drie_twee,
            model_drie_drie,
            model_drie_vier,
            ]


In [74]:
for model in modellen:
    print(summary(model, input_size=(64, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_een_drie                             [64, 10]                  --
├─Sequential: 1-1                        [64, 32, 14, 14]          --
│    └─Conv2d: 2-1                       [64, 32, 28, 28]          320
│    └─ReLU: 2-2                         [64, 32, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 32, 14, 14]          --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-4                      [64, 6272]                --
│    └─Linear: 2-5                       [64, 3136]                19,672,128
│    └─ReLU: 2-6                         [64, 3136]                --
│    └─Linear: 2-7                       [64, 1568]                4,918,816
│    └─ReLU: 2-8                         [64, 1568]                --
│    └─Linear: 2-9                       [64, 10]                  15,690
Total params: 24,606,954
Trainable params: 24,606,954
Non-trainab

In [ ]:
gin.parse_config_file("model.gin")

In [23]:
datadir =  Path("/home/admindme/code/ML22_opdracht1/data/raw/FashionMNIST")
train_dataloader, test_dataloader = get_MNIST(datadir, 64)
len(train_dataloader), len(test_dataloader)

(938, 157)

In [48]:
#models = [model_twee_twee, model_twee_vier, model_drie_twee, model_drie_vier]
#models = [model_twee_twee, model_drie_twee]

models = [model_twee_twee_channels_32_64, model_twee_drie_16_32]

epochs = 50
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = Accuracy()

for model in models:
    model = trainloop(
                epochs=epochs,
                model=model,
                optimizer=torch.optim.Adam,
                learning_rate= 0.001,
                loss_fn=loss_fn,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                log_dir="../../models/test/",
                train_steps=len(train_dataloader),
                eval_steps=150,
                #patience= ,
                #factor= ,
                #tunewriter= ,
                )

2022-12-18 13:32:30.559 | INFO     | __main__:dir_add_timestamp:9 - Logging to ../../models/test/20221218-1332
100%|██████████| 938/938 [00:41<00:00, 22.82it/s]
2022-12-18 13:33:14.581 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.1789 test 0.2868 metric ['0.8984']
100%|██████████| 938/938 [00:41<00:00, 22.63it/s]
2022-12-18 13:33:58.779 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.1514 test 0.2516 metric ['0.9153']
100%|██████████| 938/938 [00:41<00:00, 22.55it/s]
2022-12-18 13:34:43.057 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.1248 test 0.2642 metric ['0.9181']
100%|██████████| 938/938 [00:41<00:00, 22.39it/s]
2022-12-18 13:35:27.658 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.1048 test 0.2622 metric ['0.9139']
100%|██████████| 938/938 [00:42<00:00, 22.25it/s]
2022-12-18 13:36:12.545 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.0835 test 0.2512 metric ['0.9273']
100%|██████████| 938/938 [00:42<00:00, 22.14it/s]
2022-12-18 13:36:57.662 | INF

In [46]:
#model = model_een_drie
#model = model_twee_twee_channels_16_32
#model = model_twee_twee_channels_32_64
model = model_twee_drie_16_32
#model = model_twee_drie_32_64

epochs = 3
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = Accuracy()

model = trainloop(
            epochs=epochs,
            model=model,
            optimizer=torch.optim.Adam,
            learning_rate= 0.001,
            loss_fn=loss_fn,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            log_dir="../../models/test/",
            train_steps=len(train_dataloader),
            eval_steps=150,
            #patience= ,
            #factor= ,
            #tunewriter= ,
            )

2022-12-18 13:10:47.276 | INFO     | __main__:dir_add_timestamp:9 - Logging to ../../models/test/20221218-1310
100%|██████████| 938/938 [00:29<00:00, 31.94it/s]
2022-12-18 13:11:19.136 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.2058 test 0.2545 metric ['0.9094']
100%|██████████| 938/938 [00:29<00:00, 31.58it/s]
2022-12-18 13:11:51.229 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.1761 test 0.2759 metric ['0.9067']
100%|██████████| 938/938 [00:29<00:00, 31.32it/s]
2022-12-18 13:12:23.521 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.1493 test 0.2597 metric ['0.9144']
100%|██████████| 3/3 [01:36<00:00, 32.03s/it]
